In [1]:
%cd ..

/home/eli/AnacondaProjects/epych


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import collections
import glob
import functools
import logging
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import quantities as pq

import epych
from epych.statistics import alignment, grand

In [4]:
%matplotlib inline

In [5]:
logging.basicConfig(level=logging.INFO)

In [6]:
CONDITIONS = ["go_gloexp", "go_seqctl", "lo_gloexp", "lonaive", "lo_rndctl", "igo_seqctl"]
PRETRIAL_SECONDS = 0.5
POSTTRIAL_SECONDS = 0.5

In [7]:
NWB_SUBJECTS = glob.glob('/mnt/data/000253/sub-*/')

In [8]:
PILOT_FILES = []

In [9]:
aligner = epych.statistics.alignment.AlignmentSummary.unpickle("/mnt/data/000253/visual_alignment")

In [10]:
cats = {}

In [11]:
for cond in CONDITIONS:
    cats[cond] = epych.recording.Sampling.unpickle("/mnt/data/000253/grandcat_%s" % cond)
    logging.info("Loaded cross-subject LFPs for condition %s" % cond)

INFO:root:Loaded cross-subject LFPs for condition go_gloexp
INFO:root:Loaded cross-subject LFPs for condition go_seqctl
INFO:root:Loaded cross-subject LFPs for condition lo_gloexp
INFO:root:Loaded cross-subject LFPs for condition lonaive
INFO:root:Loaded cross-subject LFPs for condition lo_rndctl
INFO:root:Loaded cross-subject LFPs for condition igo_seqctl


In [12]:
PVALUE = 1e-2

In [13]:
CONTRASTS = [("logo", "lo_gloexp", "go_gloexp"), ("go_contrast", "go_gloexp", "go_seqctl"), ("lo_contrast", "lo_gloexp", "lo_rndctl"), ("ssa", "lo_gloexp", "igo_seqctl"), ("dd", "lo_rndctl", "lonaive")]

In [14]:
def initialize_grand_cluster_test(cond, key, signals):
    area = os.path.commonprefix([loc for loc in signals[0].channels.location])
    return epych.statistics.grand.GrandNonparametricClusterTest(aligner.stats[area], alpha=PVALUE)

In [15]:
summaries = {}

In [16]:
def location_prefix_signal_key(probe, signals):
    return alignment.location_prefix(probe, signals[0])

In [17]:
contrasts = {}

In [18]:
for (contrast, condl, condr) in CONTRASTS:
    summaries[contrast] = epych.statistic.Summary(location_prefix_signal_key, functools.partial(initialize_grand_cluster_test, cond))
    contrast_signals = {k: (cats[condl].signals[k], cats[condr].signals[k]) for k in cats[condl].signals.keys()}
    summaries[contrast].calculate([contrast_signals])
    contrasts[contrast] = {k: v.result() for k, v in summaries[contrast].stats.items()}
    logging.info("Nonparametrically contrasted for %s" % contrast)

KeyboardInterrupt: 

In [ ]:
area_titles = {
    "VISal": "AL",
    "VISam": "AM",
    "VISl": "LM",
    "VISp": "V1",
    "VISpm": "PM",
    "VISrl": "RL",
}
def sigtitle(name, signal):
    if name in area_titles:
        return area_titles[name]
    return name

In [ ]:
ADAPTED_ONSET = pq.Quantity(-1.0) * pq.second
ADAPTED_OFFSET = pq.Quantity(-0.5) * pq.second
ODDBALL_ONSET = pq.Quantity(-1.9017372477960602e-14) * pq.second
ODDBALL_OFFSET = pq.Quantity(0.5004545430388676) * pq.second
EVENTS = {
    "Onset": (ODDBALL_ONSET.magnitude, 'lightgreen'),
    "Offset": (ODDBALL_OFFSET.magnitude, 'red'),
}

In [ ]:
def compute_contrast(condl, condr):
    contrast = epych.statistics.grand.summary_t_test(summaries[condl], summaries[condr])
    lfp_diffs, csd_diffs = {}, {}
    for area in sorted(contrast.keys()):
        meanl, meanr = summaries[condl].stats[area].mean, summaries[condr].stats[area].mean
        diff_data = (meanl.data - meanr.data) * (contrast[area][1] < PVALUE)
        lfp_diffs[area] = epych.signals.lfp.EvokedLfp(meanl.channels, diff_data, meanl.dt, meanl.times)

        csdl, csdr = meanl.downsample(4).current_source_density(depth_column="vertical"), meanr.downsample(4).current_source_density(depth_column="vertical")
        csdl._data, csdr._data = csdl.data.rescale('nA / mm**2'), csdr.data.rescale('nA / mm**2')
        diff_data = (csdl.data - csdr.data)* (contrast[area][1][::4, :, :] < PVALUE)
        csd_diffs[area] = epych.signals.lfp.EvokedLfp(meanl.channels[::4], diff_data, meanl.dt, meanl.times)

    lfps = epych.recording.EvokedSampling(epych.recording.empty_intervals(), epych.recording.empty_trials(), epych.recording.default_units(),
                                          **lfp_diffs)
    csds = epych.recording.EvokedSampling(epych.recording.empty_intervals(), epych.recording.empty_trials(), epych.recording.default_units(),
                                          **csd_diffs)
    return contrast, lfps, csds

In [ ]:
def plot_contrast(name, condl, condr):
    contrast, lfps, csd = compute_contrast(condl, condr)
    lfps.plot(vmin=-1e-1, vmax=1e-1, figure=name + "_all_lfps.png", sigtitle=sigtitle, **EVENTS)
    csd.plot_signals(name + "/significant", cmap='PiYG', sigtitle=sigtitle, **EVENTS)

In [ ]:
for (contrast, condl, condr) in CONTRASTS:
    plot_contrast(contrast, condl, condr)
    logging.info("Plotted grand t-test contrasts for %s" % contrast)

In [ ]:
OFFSET = np.array(0.25) * pq.second

def subtraction_variance(cond_summary):
    result = cond_summary.__class__(cond_summary.signal_key, cond_summary.stat)
    for area, grand_var in cond_summary.stats.items():
        alignment = grand_var.alignment
        mean, variance = grand_var.mean, grand_var.result()
        mean_subtraction = mean[(ODDBALL_ONSET - OFFSET).magnitude:(ODDBALL_OFFSET + OFFSET).magnitude] - mean[(ADAPTED_ONSET - OFFSET).magnitude:(ADAPTED_OFFSET + OFFSET).magnitude]
        mean_subtraction = mean_subtraction.shift_timestamps(-OFFSET)
        variance_addition = variance[(ODDBALL_ONSET - OFFSET).magnitude:(ODDBALL_OFFSET + OFFSET).magnitude] + variance[(ADAPTED_ONSET - OFFSET).magnitude:(ADAPTED_OFFSET + OFFSET).magnitude]
        variance_addition = variance_addition.shift_timestamps(-OFFSET)
        result._stats[area] = grand.GrandVariance(alignment, mean_subtraction)
        result.stats[area].data["diffs"] = variance_addition.data * (grand_var.data["n"] - 1)
        result.stats[area].data["k"] = grand_var.data["k"]
        result.stats[area].data["n"] = grand_var.data["n"]
    return result

In [ ]:
CONTRAST_STDS = {}

In [ ]:
def compute_subtraction_contrast(name, condl, condr):
    summaryl = subtraction_variance(summaries[condl])
    summaryr = subtraction_variance(summaries[condr])
    contrast = epych.statistics.grand.summary_t_test(summaryl, summaryr)
    lfp_diffs, csd_diffs = {}, {}
    for area in sorted(contrast.keys()):
        meanl, meanr = summaryl.stats[area].mean, summaryr.stats[area].mean
        diff_data = (meanl.data - meanr.data) * (contrast[area][1] < PVALUE)
        lfp_diffs[area] = epych.signals.lfp.EvokedLfp(meanl.channels, diff_data, meanl.dt, meanl.times)

        csdl, csdr = meanl.downsample(4).current_source_density(depth_column="vertical"), meanr.downsample(4).current_source_density(depth_column="vertical")
        csdl._data, csdr._data = csdl.data.rescale('nA / mm**2'), csdr.data.rescale('nA / mm**2')
        diff_data = (csdl.data - csdr.data) * (contrast[area][1][::4, :, :] < PVALUE)
        if name not in CONTRAST_STDS:
            CONTRAST_STDS[name] = {}
        CONTRAST_STDS[name][area] = diff_data.std()
        csd_diffs[area] = epych.signals.lfp.EvokedLfp(meanl.channels[::4], diff_data, meanl.dt, meanl.times)

    lfps = epych.recording.EvokedSampling(epych.recording.empty_intervals(), epych.recording.empty_trials(), epych.recording.default_units(),
                                          **lfp_diffs)
    csds = epych.recording.EvokedSampling(epych.recording.empty_intervals(), epych.recording.empty_trials(), epych.recording.default_units(),
                                          **csd_diffs)
    return contrast, lfps, csds

In [ ]:
def plot_subtraction_contrast(name, condl, condr):
    contrast, lfps, csd = compute_subtraction_contrast(name, condl, condr)
    lfps.plot(vmin=-1e-1, vmax=1e-1, figure=name + "_4minus3_all_lfps.svg", sigtitle=sigtitle, **EVENTS)
    csd.plot_signals(name + "_4minus3_significant", vmaxs={area: 2 * std_dev for area, std_dev in CONTRAST_STDS[name].items()}, vmins={area: -2 * std_dev for area, std_dev in CONTRAST_STDS[name].items()},
                     sigtitle=sigtitle, cmap='PiYG', **EVENTS)

In [ ]:
for (contrast, condl, condr) in CONTRASTS:
    plot_subtraction_contrast(contrast, condl, condr)
    logging.info("Plotted grand t-test of 4th-minus-3rd contrasts for %s" % contrast)